In [1]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

import os
import sys
import time
import datetime
import argparse
import tqdm

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

In [3]:
def evaluate(model, path, iou_thres, conf_thres, nms_thres, img_size, batch_size):
    model.eval()

    # Get dataloader
    dataset = ListDataset(path, img_size=img_size, augment=False, multiscale=False)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=dataset.collate_fn
    )

    Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

    labels = []
    sample_metrics = []  # List of tuples (TP, confs, pred)
    for batch_i, (_, imgs, targets) in enumerate(tqdm.tqdm(dataloader, desc="Detecting objects")):

        # Extract labels
        labels += targets[:, 1].tolist()
        # Rescale target
        targets[:, 2:] = xywh2xyxy(targets[:, 2:])
        targets[:, 2:] *= img_size

        imgs = Variable(imgs.type(Tensor), requires_grad=False)

        with torch.no_grad():
            outputs = model(imgs)
            outputs = non_max_suppression(outputs, conf_thres=conf_thres, nms_thres=nms_thres)

        sample_metrics += get_batch_statistics(outputs, targets, iou_threshold=iou_thres)

    # Concatenate sample statistics
    true_positives, pred_scores, pred_labels = [np.concatenate(x, 0) for x in list(zip(*sample_metrics))]
    precision, recall, AP, f1, ap_class = ap_per_class(true_positives, pred_scores, pred_labels, labels)

    return precision, recall, AP, f1, ap_class

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
data_config = parse_data_config('config/coco.data')
valid_path = data_config["valid"]
class_names = load_classes(data_config["names"])

In [6]:
data_config

{'gpus': '0,1,2,3',
 'num_workers': '10',
 'classes': '80',
 'train': 'data/coco/trainvalno5k.txt',
 'valid': 'data/coco/5k.txt',
 'names': 'data/coco.names',
 'backup': 'backup/',
 'eval': 'coco'}

In [24]:
model = Darknet("config/yolov3_skymul.cfg").to(device)
if "weights/yolov3.weights".endswith(".weights"):
    # Load darknet weights
    model.load_darknet_weights("weights/yolov3.weights")
else:
    # Load checkpoint weights
    model.load_state_dict(torch.load("weights/yolov3.weights"))

In [10]:
path = valid_path
iou_thres = 0.5
conf_thres = 0.001
nms_thres = 0.5
img_size = 416
batch_size = 8

In [25]:
model.eval()

Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_3): BatchNorm2d(64, eps=1e-05, momen

In [26]:
model.yolo_layers

[YOLOLayer(
   (mse_loss): MSELoss()
   (bce_loss): BCELoss()
 ), YOLOLayer(
   (mse_loss): MSELoss()
   (bce_loss): BCELoss()
 ), YOLOLayer(
   (mse_loss): MSELoss()
   (bce_loss): BCELoss()
 )]

In [30]:
model.yolo_layers[-1].img_dim

416

In [42]:
(list(model.parameters())[-1])

Parameter containing:
tensor([ 0.0120,  0.0469,  0.0411,  0.0488, -0.0120, -0.0721, -0.0239,  0.0041,
        -0.0558, -0.0109, -0.0199, -0.0270, -0.0067,  0.0114,  0.0151,  0.0157,
         0.0334,  0.0311], device='cuda:0', requires_grad=True)

In [31]:
model_config = parse_model_config('config/yolov3_skymul.cfg')

In [37]:
[layer for layer in model_config if layer['type'] == 'yolo']

[{'type': 'yolo',
  'mask': '6,7,8',
  'anchors': '10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326',
  'classes': '1',
  'num': '9',
  'jitter': '.3',
  'ignore_thresh': '.7',
  'truth_thresh': '1',
  'random': '1'},
 {'type': 'yolo',
  'mask': '3,4,5',
  'anchors': '10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326',
  'classes': '1',
  'num': '9',
  'jitter': '.3',
  'ignore_thresh': '.7',
  'truth_thresh': '1',
  'random': '1'},
 {'type': 'yolo',
  'mask': '0,1,2',
  'anchors': '10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326',
  'classes': '1',
  'num': '9',
  'jitter': '.3',
  'ignore_thresh': '.7',
  'truth_thresh': '1',
  'random': '1'}]

In [39]:
data_config = parse_data_config("config/rebar.data")
train_path = data_config["train"]
valid_path = data_config["valid"]
class_names = load_classes(data_config["names"])

In [40]:
class_names

['intersection']

In [12]:
dataset = ListDataset(path, img_size=img_size, augment=False, multiscale=False)

In [13]:
dataset

In [14]:
dataset.img_files

['/c/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000164.jpg\n',
 '/c/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000192.jpg\n',
 '/c/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000283.jpg\n',
 '/c/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000397.jpg\n',
 '/c/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000589.jpg\n',
 '/c/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000599.jpg\n',
 '/c/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000711.jpg\n',
 '/c/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000757.jpg\n',
 '/c/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000764.jpg\n',
 '/c/Users/Trace/Documents/S

In [15]:
path

'data/coco/5k.txt'

In [19]:
[img_file.replace('/c/', 'C:/') for img_file in dataset.img_files]

['C:/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000164.jpg\n',
 'C:/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000192.jpg\n',
 'C:/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000283.jpg\n',
 'C:/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000397.jpg\n',
 'C:/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000589.jpg\n',
 'C:/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000599.jpg\n',
 'C:/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000711.jpg\n',
 'C:/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000757.jpg\n',
 'C:/Users/Trace/Documents/SkyMul/PyTorch-YOLOv3/data/coco/images/val2014/COCO_val2014_000000000764.jpg\n',
 'C:/Users/Trace/Documents/S

In [43]:
import glob

In [45]:
train = glob.glob('data/rebar/images/train/*')
val = glob.glob('data/rebar/images/val/*')

In [49]:
with open('data/rebar/train.txt', 'w') as f:
    for img in train:
        f.write(img.replace('\\', '/') + '\n')

In [51]:
with open('data/rebar/valid.txt', 'w') as f:
    for img in val:
        f.write(img.replace('\\', '/') + '\n')

In [57]:
import pandas as pd

In [84]:
df = pd.read_csv('data/rebar/annotations.csv', names=['path', 'x', 'y', 'w', 'h', 'label'])
df['filename'] = df['path'].apply(lambda x: x.split('\\')[-1])
df['class'] = 0
path = 'data/rebar/labels/train/'
for img in df['filename'].unique():
    (df[df['filename']==img][['class', 'x', 'y', 'w', 'h']]).to_csv(path + img.replace('jpg', 'txt'), sep=' ', header=False, index=False)

In [86]:
df = pd.read_csv('data/rebar/annotations_val.csv', names=['path', 'x', 'y', 'w', 'h', 'label'])
df['filename'] = df['path'].apply(lambda x: x.split('\\')[-1])
df['class'] = 0
path = 'data/rebar/labels/val/'
for img in df['filename'].unique():
    (df[df['filename']==img][['class', 'x', 'y', 'w', 'h']]).to_csv(path + img.replace('jpg', 'txt'), sep=' ', header=False, index=False)

In [92]:
def get_labels(annotations_path, output_path):
    df = pd.read_csv(annotations_path, names=['path', 'x', 'y', 'w', 'h', 'label'])
    df['filename'] = df['path'].apply(lambda x: x.split('\\')[-1])
    df['class'] = 0
    df[['x', 'w']] /= 640
    df[['y', 'h']] /= 480
    for img in df['filename'].unique():
        (df[df['filename']==img][['class', 'x', 'y', 'w', 'h']]).to_csv(output_path + img.replace('jpg', 'txt'), 
                                                                        sep=' ', header=False, index=False)

In [93]:
get_labels('data/rebar/annotations.csv', 'data/rebar/labels/train/')

In [94]:
get_labels('data/rebar/annotations_val.csv', 'data/rebar/labels/train/')

In [79]:
(df[df['filename'] == df['filename'].unique()[0]][['x', 'y', 'w', 'h']]).to_string()

'     x    y    w    h\n0  168  235  202  265\n1  236  251  267  279\n2  297  264  328  289\n3  313  319  337  351\n4  245  311  275  341\n5  167  292  195  325\n6  152  371  194  397\n7  246  385  274  411\n8  321  391  347  417'

In [ ]:
df.to_csv